# 1. Load IMOEX index data (target benchmark)

- Load IMOEX data from https://smart-lab.ru/q/index_stocks/IMOEX/# and store to ImoexData/*.csv
- Load T-1 quotes from https://iss.moex.com/iss/engines/stock/markets/shares/securities.xml?index=IMOEX&marketprice_board=1 to StockData/*.csv

Set tickers to exclude (e.g. not applicable to IIA) in file
```
exclude.csv
```

In [1]:
import time
import pandas as pd
from IPython.display import display
import requests
import bs4
import os

class Imoex:

    index = pd.DataFrame(columns=[ 'percent', 'price', 'name'],index=['ticker'])

    def __init__(self):

        timestr = time.strftime("%Y%m%d")
        fname = f'ImoexData/Imoex{timestr}.csv'

        self.fetchToFile(fname)

        self.index = pd.read_csv(fname,index_col='ticker')

    def getIndex(self):
        return self.index

    def fetchToFile(self, fname):
        if not(os.path.exists(fname)):
            url = 'https://smart-lab.ru/q/index_stocks/IMOEX/#'
            response = requests.get(url)

            soup = bs4.BeautifulSoup(response.text, 'html.parser')
            table = soup.find_all('table')[0]
            rows = table.find_all('tr')

            index = pd.DataFrame(columns=[ 'percent', 'price', 'name'],index=['ticker'])

            for row in rows[1:]:
                cells = row.find_all('td')

                name = cells[1].text.strip()

                percent = float(cells[2].text.strip().strip('%'))

                ticker_href = cells[3].find_all('a')[0].get('href')
                ticker = ticker_href[9:]

                price = cells[5].text.strip()

                index.loc[ticker] = [percent,price,name]

            index = index.dropna()

            excluded = pd.read_csv('exclude.csv',index_col=0)
            index = index.drop(excluded.index,errors='ignore')

            index.to_csv(fname,index_label='ticker')

        # display(df)

imoex = Imoex()
tgtPercentage = imoex.getIndex()
tgtPercentage

,percent,price,name
ticker,,,
LKOH,16.19,6964.0000,ЛУКОЙЛ
SBER,12.10,257.1900,Сбербанк
GAZP,11.77,135.6200,ГАЗПРОМ ао
TATN,6.09,596.0000,Татнфт 3ао
YDEX,4.16,3971.5000,ЯНДЕКС
GMKN,3.74,104.2400,ГМКНорНик
TCSG,3.70,2567.5000,ТКСХолд ао
NVTK,3.37,958.2000,Новатэк ао
PLZL,3.00,13627.5000,Полюс


In [2]:
pd.read_csv('exclude.csv',index_col=0)

""
ticker
OZON
FIVE
AGRO
GLTR


In [3]:
# Helper classes

## Moex class

In [4]:
import requests
import os
import time
import pandas as pd
from IPython.display import display


class Moex:

    # ticker, lot_aty, price, date, name
    quotes = pd.DataFrame()

    def __init__(self,forceFetch = False):
        # need to update only onece a day
        timestr = time.strftime("%Y%m%d")
        fname = f'StockData/StockData{timestr}.csv'
        self.__fetchMoexToFile(fname,forceFetch)
        self.quotes = pd.read_csv(fname)
        self.quotes = self.quotes.rename(columns={\
            'SECID':'ticker',\
            'LOTSIZE':'lotSize',\
            'PREVPRICE':'price',\
            'PREVDATE':'date',\
            'LATNAME':'name'})
        self.quotes= self.quotes.set_index('ticker')
        self.quotes = self.quotes.drop(columns=['BOARDID','SHORTNAME','FACEVALUE','STATUS','BOARDNAME',\
                                                'DECIMALS','SECNAME','REMARKS','MARKETCODE','INSTRID','SECTORID',\
                                                'MINSTEP','PREVWAPRICE','FACEUNIT','ISSUESIZE','ISIN',\
                                                'REGNUMBER','PREVLEGALCLOSEPRICE','CURRENCYID','SECTYPE','LISTLEVEL','SETTLEDATE'])

    # get stocks quotes T-1 using MOEX API
    def __fetchMoexToFile(self, fname,forceFetch=False):
        if not(os.path.exists(fname)) or forceFetch:
            url = 'https://iss.moex.com/iss/engines/stock/markets/shares/securities.xml?index=IMOEX&marketprice_board=1'
            response = requests.get(url)
            df = pd.read_xml(response.text,xpath='//data[@id="securities"]/rows/row').set_index('SECID')
            df.to_csv(fname)

## Portfolio class

In [5]:
import math
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
from IPython.display import display

'''
tgt - target
chg - change (delta)
'''
class Portfolio:

    tolerance = 0
    
    def __init__(self,tolerance=0):
        pd.options.display.float_format = '{:,.2f}'.format
        self.tolerance = tolerance
    
    def setTarget(self,percentages):
        """
        set target percentage of stocks in Portfolio\n
        - percentages -  DataFrame {ticker: string,index; percent: 0..1 float}
        """
        self.target = percentages
        totalPct = self.target['percent'].sum()
        self.target['percent'] = self.target['percent']/totalPct
    
    def getTarget(self):
        return self.target
    
    def setCurrent(self,path):
        
        df = pd.read_csv(path,index_col='ticker')
        df['qty']=df.sum(axis=1)
        
        self.current=df[['qty']]

        self.cash = self.current.loc['@RUB','qty']

        # get current quotes
        moex = Moex()
        self.current = self.current.join(moex.quotes,how='outer')
        self.current.fillna(value=0,inplace=True)
        
        self.current['percent'] = self.current['price']*self.current['qty'] / self.__getTotal()

        self.current.at['@RUB','price'] = 1
        self.current['value']=self.current['price']*self.current['qty']
  
    def getCurrent(self):
        return self.current
     
    def __getTotal(self):
        return (self.current['qty']*self.current['price']).sum() + self.cash 
    
    
    @staticmethod
    def applyLots(df):
        df['lotPrice'] = df['lotSize'] * df['price']
        df['valueChgNoLots'] = df['valueTgt'] - df['value']
        df['lotQtyChg'] = (df['valueChgNoLots'] / df['lotPrice']).apply(np.round)
        df['valueChg'] = df['lotQtyChg'] * df['lotPrice']
        return df

    def rebalance(self,verbouse = False):
               
        # combine all data together
        df = self.target.join(self.current,lsuffix='Tgt',how='outer')

        df['percentTgt'] = df['percentTgt'] / df['percentTgt'].sum()
        df['valueTgt'] = df['percentTgt'] * df['value'].sum()
        df = Portfolio.applyLots(df)

        df = df.fillna(0)
        cashChange = -  df['valueChg'].sum()+ df.at['@RUB','valueChg']
        df.at['@RUB','valueChg'] = cashChange
        df.at['@RUB','lotQtyChg'] = cashChange

        df = df.sort_values(by='valueChg',ascending=False)
        
        if verbouse:
            return df[['name','qty','price','value','lotSize','lotPrice','percentTgt','valueTgt','valueChg','lotQtyChg']]
        return df[['name','valueChg']]

    def getStd(self,verbouse = False):
        """ std deviation as rebalance quality criteria """
        df = self.target.join(self.current,lsuffix='Tgt')
        ttl = self.__getTotal()
        df['valueTgt'] = df['percentTgt']*ttl
        df['value'] = df['value'].fillna(0)
        df['delta'] = -df['value'] + df['valueTgt']

        std = np.std(df['delta'])
        if verbouse:
            dd = df[['value','valueTgt','delta']]
            dd['delta %'] = dd['delta'] / self.__getTotal() * 100
            dd['value'] = dd['value'] / 1000
            dd['valueTgt'] = dd['valueTgt']/1000
            dd['delta'] = dd['delta']/1000
            dd['deltaAbs']=abs(dd['delta'])
            dd = dd.sort_values(by=['deltaAbs'],ascending=False)
            display(dd)

        return std

In [6]:
#! pip install lxml

## Set Target portfolio

In [7]:
p = Portfolio(tolerance=10000)
p.setTarget(tgtPercentage)

p.getTarget()

,percent,price,name
ticker,,,
LKOH,0.17,"6,964.00",ЛУКОЙЛ
SBER,0.12,257.19,Сбербанк
GAZP,0.12,135.62,ГАЗПРОМ ао
TATN,0.06,596.00,Татнфт 3ао
YDEX,0.04,"3,971.50",ЯНДЕКС
GMKN,0.04,104.24,ГМКНорНик
TCSG,0.04,"2,567.50",ТКСХолд ао
NVTK,0.03,958.20,Новатэк ао
PLZL,0.03,"13,627.50",Полюс


# 2. Set current portfolio

```
CurrentV.csv
```

In [8]:
p.setCurrent('CurrentV.csv')
p.getCurrent()

,qty,price,lotSize,date,name,percent,value
ticker,,,,,,,
@RUB,"403,929.00",1.00,0.00,0,0,0.00,"403,929.00"
AFKS,300.00,15.16,100.00,2024-10-17,AFK Sistema,0.00,"4,547.40"
AFLT,120.00,56.25,10.00,2024-10-17,Aeroflot,0.01,"6,750.00"
AGRO,0.00,"1,212.40",1.00,2024-10-17,GDR ROS AGRO PLC ORD SHS,0.00,0.00
ALRS,150.00,54.15,10.00,2024-10-17,ALROSA ao,0.01,"8,122.50"
ASTR,5.00,537.15,1.00,2024-10-17,Astra Group,0.00,"2,685.75"
BSPB,10.00,362.00,10.00,2024-10-17,BSP,0.00,"3,620.00"
CBOM,800.00,6.32,100.00,2024-10-17,MKB ao,0.00,"5,059.20"
CHMF,12.00,"1,191.00",1.00,2024-10-17,Severstal - ao,0.01,"14,292.00"


# 3. Get change value needed to accomodate to target index and buy (manually) stocks

In [9]:
p.rebalance()

,name,valueChg
ticker,,
LKOH,LUKOIL,"55,452.00"
YDEX,YANDEX,"51,785.50"
SBER,Sberbank,"43,616.90"
GAZP,Gazprom,"42,175.50"
TATN,Tatneft-3,"21,941.00"
GMKN,NorNickel GMK,"13,546.00"
PLZL,Polus,"13,526.50"
TCSG,IPJSC TCS Holding,"12,887.50"
NVTK,NOVATEK,"12,461.80"
